# Price stock tendency prediction pipeline


## Setup

In [5]:
%load_ext autoreload
%autoreload 2

from notebook_config import setup_notebook
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

setup_notebook()

In [6]:
from tqdm.notebook import trange, tqdm

from helpers.data_helper import *

In [7]:
DATA_PATH = '../../data/'
FILE_SUFFIX = '.txt'
price_column = 'Close'

In [8]:
OUTPUT_PATH = 'results/'
N_CLASSES = 2
METRICS_PATH = f"{OUTPUT_PATH}metrics/{N_CLASSES}_classes/"
PIPELINE_LABEL = 'linear_clfs'

THRESH_DIFF = 0.005 if N_CLASSES is 3 else None

In [9]:
from datasets.stocks_data_wrapper import StocksDataWrapper
#data_wrapper = StocksDataWrapper.read_from(f"{DATA_PATH}{quotation}{FILE_SUFFIX}", compute_features=True, predict_n=1, thresh_diff=0.005, normalize=True)
#data_wrapper.head()

## Classifier and evaluation

In [11]:
from sklearn.utils.class_weight import compute_class_weight

def compute_class_weights(y_train):
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    # format as dict
    return {c:w for c,w in zip(np.unique(y_train), class_weights)}

### Grid Search

In [10]:
from sklearn.model_selection import TimeSeriesSplit
# build the time series split for cross validation so no data leakage
tscv = TimeSeriesSplit(n_splits=5)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

N_JOBS=-1

# default parameters
best_params = {'DTree': {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1}, 'RandomForest': {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 15, 'n_estimators': 50}, 'SVM': {'C': 0.5, 'gamma': 'scale', 'kernel': 'poly'}, 'LR': {'C': 0.1, 'fit_intercept': False, 'penalty': 'l2'}}
param_grid = {
    'DTree':{
        'criterion':('entropy', 'gini'),
        'max_depth':[3,5,8,10],
        'min_samples_leaf':[1,2,3,5,10,15],
    },
    'RandomForest':{
        'criterion':('entropy', 'gini'),
        'n_estimators':[100,200,500],
        'max_depth':[3,5,8,10],
        'min_samples_leaf':[3,5,10,15],
        'max_features':['auto', 'sqrt', 'log2'], 
    },
    'SVM':{
        #'C':[1e-3, 1e-2, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0, 100.0],
        'C':[0.1, 1.0, 10.0, 50.0, 100.0, 200.0],
        'probability':[True, False],
        'kernel':('linear', 'poly', 'rbf'),
        'gamma':('scale', 'auto'),
    },
    'LR':
    {
        'max_iter':[1000],
        'penalty':('none', 'l2'),
        'C':[0.1, 1.0, 10.0, 50.0, 100.0, 200.0],
        'fit_intercept':[True, False],
    },   
    
}

"""param_grid = {
    'DTree':{
        'criterion':('entropy', 'gini'),
    },
    'RandomForest':{
        'criterion':('entropy', 'gini'),
    },
    'SVM':{
        #'C':[1e-3, 1e-2, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0, 100.0],
        'C':[0.1, 1.0, 10.0],
        'gamma':('scale', 'auto'),
    },
    'LR':
    {
        'penalty':('none', 'l2'),
        'fit_intercept':[True, False],
    },   
    
}"""

def compute_grid_search(param_grid, X_train, y_train, n_splits=5, class_weights=None):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    best_params = {}
    
    print("Performing grid search...")
    dtree = DecisionTreeClassifier(random_state=42, class_weight=class_weights)
    clf = GridSearchCV(dtree, param_grid['DTree'], cv=tscv.split(X_train), n_jobs=N_JOBS, verbose=2)
    clf.fit(X_train, y_train)
    best_params['DTree'] = clf.best_params_

    random_forest = RandomForestClassifier(random_state=42, class_weight=class_weights)
    clf = GridSearchCV(random_forest, param_grid['RandomForest'], cv=tscv.split(X_train), n_jobs=N_JOBS, verbose=32)
    clf.fit(X_train, y_train)
    best_params['RandomForest'] = clf.best_params_

    
    svm = SVC(random_state=42, class_weight=class_weights, cache_size=4096)
    clf = GridSearchCV(svm, param_grid['SVM'], cv=tscv.split(X_train), n_jobs=N_JOBS, verbose=32)
    clf.fit(X_train, y_train)
    best_params['SVM'] = clf.best_params_
    
    lr = LogisticRegression(random_state=42, class_weight=class_weights)
    clf = GridSearchCV(lr, param_grid['LR'], cv=tscv.split(X_train), n_jobs=N_JOBS, verbose=2)
    clf.fit(X_train, y_train)
    best_params['LR'] = clf.best_params_
    return best_params


### Classifiers

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.SVR import SVC

"""clf_dict = {}

clf_dict['DTree'] = DecisionTreeClassifier(**best_params['DTree'], random_state=42, class_weight=class_weights)
clf_dict['RandomForest'] = RandomForestClassifier(**best_params['RandomForest'], random_state=42, class_weight=class_weights)
clf_dict['SVM'] = SVC(**best_params['SVM'], class_weight=class_weights)
clf_dict['LR'] = LogisticRegression(**best_params['LR'], class_weight=class_weights)"""

"clf_dict = {}\n\nclf_dict['DTree'] = DecisionTreeClassifier(**best_params['DTree'], random_state=42, class_weight=class_weights)\nclf_dict['RandomForest'] = RandomForestClassifier(**best_params['RandomForest'], random_state=42, class_weight=class_weights)\nclf_dict['SVM'] = SVC(**best_params['SVM'], class_weight=class_weights)\nclf_dict['LR'] = LogisticRegression(**best_params['LR'], class_weight=class_weights)"

### Training

In [13]:
from sklearn.metrics import accuracy_score, plot_confusion_matrix, f1_score

def train_evaluate_classifiers(clf_dict, X_train, y_train, X_test, y_test):
    clf_metrics = {}
    for clf_label, clf in clf_dict.items():
        clf.fit(X_train, y_train)
        #y_pred = clf.predict(X_train)
        #print(f"Training accuracy : {accuracy_score(y_pred, y_train)}")
        y_pred = clf.predict(X_test)

        accuracy = accuracy_score(y_pred, y_test)
        f1 = f1_score(y_pred, y_test, average='weighted')

        #print(f"[{clf_label}] Accuracy : {accuracy:.2f}, F1 : {f1:.2f}")
        #plot_confusion_matrix(clf, X_test, y_test, normalize='true')
        clf_metrics[clf_label] = {'acc':accuracy, 'f1':f1}
        
    return clf_metrics

### Pipeline

In [14]:
USE_CLASS_WEIGHTS = True
PERFORM_GRID_SEARCH = True
MAX_SIZE = 5000
n_splits=5

quotations = ['AAL', 'AAPL', 'AMZN','CMCSA', 'COST', 'GM', 'GOOG', 'IBM', 'JNJ', 'KO','PEP', 'TSLA', 'WMT', 'XOM']
days_predict_feature_set = {
    1:['Volume', 'Close', 'LowLen', 'Difference', 'BG_L_Band', 'GAP', 'MACD_diff'],
    5:['Volume', 'Close', 'LowLen', 'Difference', 'SMA(20) - SMA(10)', 'BG_H_Band_Indicator', 'MACD'],
    10:['Volume', 'Close', 'BodyLen', 'Difference', 'SMA(20) - SMA(10)', 'EMA_Diff', 'MACD_diff'],
    20:['Volume', 'Close', 'LowLen', 'Difference', 'EMA(14)', 'BG_H_Band', 'MACD_diff'],
    50:['Volume', 'Close', 'LowLen', 'Difference', 'BG_L_Band','GAP','MACD_diff'],
}

#quotations=['CMCSA']
#days_predict_feature_set = {1:days_predict_feature_set[1]}

In [15]:
predict_n_metrics_dict = {}

In [16]:
for predict_n, features_list in tqdm(days_predict_feature_set.items()):
    quot_metrics_dict = {}
    for quot in tqdm(quotations):
        data_wrapper = StocksDataWrapper.read_from(f"{DATA_PATH}{quot}{FILE_SUFFIX}", compute_features=True, 
                                                   predict_n=predict_n, thresh_diff=None, normalize=True)


        X_train, X_test, y_train, y_test = data_wrapper.get_datasets(n_splits=1, val_size=0.2, 
                                                                     y_column='Next', 
                                                                     features_list=features_list)
        
        class_weights, grid_search_class_weights = None, None
        if USE_CLASS_WEIGHTS:
            class_weights=compute_class_weights(y_train)
            grid_search_class_weights = compute_class_weights(y_train[-MAX_SIZE:])



        if PERFORM_GRID_SEARCH:
            # ceil the number of samples, get the lasts MAX_SIZE samples 
            best_params = compute_grid_search(param_grid, 
                                              X_train=X_train[-MAX_SIZE:],
                                              y_train=y_train[-MAX_SIZE:],
                                              class_weights=grid_search_class_weights, 
                                              n_splits=n_splits)
            params_df = pd.DataFrame.from_dict(best_params)
            params_df.to_csv(f"grid_search_results/{predict_n}_{quot}.csv", index=False)

        clf_dict = {}

        clf_dict['DTree'] = DecisionTreeClassifier(**best_params['DTree'], random_state=42, class_weight=class_weights)
        clf_dict['RandomForest'] = RandomForestClassifier(**best_params['RandomForest'], random_state=42, class_weight=class_weights)
        clf_dict['SVM'] = SVC(**best_params['SVM'], class_weight=class_weights)
        clf_dict['LR'] = LogisticRegression(**best_params['LR'], class_weight=class_weights)

        
        quot_metrics_dict[quot] = train_evaluate_classifiers(clf_dict, X_train, y_train, X_test, y_test)
        
    metrics_df = pd.DataFrame.from_dict(quot_metrics_dict).T
    acc_df = metrics_df.applymap(lambda metrics: metrics['acc'])
    f1_df = metrics_df.applymap(lambda metrics: metrics['f1'])

    acc_df.to_csv(f"metrics/acc_{predict_n}.csv", index=True)
    f1_df.to_csv(f"metrics/f1_{predict_n}.csv", index=True)
        
    predict_n_metrics_dict[predict_n] = quot_metrics_dict.copy()
    


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

Performing grid search...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Performing grid search...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Performing grid search...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


d:\prog\tm\stonksmarketprediction\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
d:\prog\tm\stonksmarketprediction\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


Performing grid search...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Performing grid search...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Performing grid search...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Performing grid search...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for 

...

Performing grid search...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Performing grid search...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


d:\prog\tm\stonksmarketprediction\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
d:\prog\tm\stonksmarketprediction\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


Performing grid search...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


d:\prog\tm\stonksmarketprediction\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
d:\prog\tm\stonksmarketprediction\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


Performing grid search...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [17]:
print(X_train.shape)
X_train = X_train[-5000:]
print(X_train.shape)
cv = tscv.split(X_train)
for split in cv:
    train_ids, test_ids = split
    train = [X_train[i] for i in train_ids]
    val = [X_train[i] for i in test_ids]
    print(np.where(np.isinf(train)))
    print(np.where(np.isnan(train)))
    print(np.where(np.isinf(val)))
    print(np.where(np.isnan(val)))

(2753, 7)
(2753, 7)
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [18]:
params_df = pd.DataFrame.from_dict(best_params)
params_df.to_csv(f"results/grid_search/{N_CLASSES}_classes/{predict_n}_{quot}.csv", index=False)

In [19]:
print(predict_n_metrics_dict)

{1: {'AAL': {'DTree': {'acc': 0.5207439198855508, 'f1': 0.5342762796807659}, 'RandomForest': {'acc': 0.5007153075822603, 'f1': 0.5320477382211538}, 'SVM': {'acc': 0.47067238912732473, 'f1': 0.5630758763862446}, 'LR': {'acc': 0.47782546494992845, 'f1': 0.4817446940781028}}, 'AAPL': {'DTree': {'acc': 0.48068669527896996, 'f1': 0.5666769203237425}, 'RandomForest': {'acc': 0.4563662374821173, 'f1': 0.6055698033952478}, 'SVM': {'acc': 0.44778254649499283, 'f1': 0.6185770750988142}, 'LR': {'acc': 0.44778254649499283, 'f1': 0.6185770750988142}}, 'AMZN': {'DTree': {'acc': 0.5321888412017167, 'f1': 0.6391185845316079}, 'RandomForest': {'acc': 0.5464949928469242, 'f1': 0.7057419149068512}, 'SVM': {'acc': 0.4449213161659514, 'f1': 0.46273426123452444}, 'LR': {'acc': 0.5436337625178826, 'f1': 0.6426906640823297}}, 'CMCSA': {'DTree': {'acc': 0.5269749518304432, 'f1': 0.6699144549449183}, 'RandomForest': {'acc': 0.5394990366088632, 'f1': 0.6720614738954593}, 'SVM': {'acc': 0.5322736030828517, 'f1': 

In [26]:
save_dict(dict_save=predict_n_metrics_dict, path=f"{METRICS_PATH}{PIPELINE_LABEL}_dict.txt")

In [21]:
#print(quot_metrics_dict)

for quot, classifiers in quot_metrics_dict.items():
    print(f"[{quot}]")
    for label, metrics in classifiers.items():
        print(f"\t[{label}] : acc :  {metrics['acc']:.2f}, f1 : {metrics['f1']:.2f}")
    
    print("\n")


[AAL]
	[DTree] : acc :  0.52, f1 : 0.52
	[RandomForest] : acc :  0.52, f1 : 0.52
	[SVM] : acc :  0.51, f1 : 0.51
	[LR] : acc :  0.51, f1 : 0.51


[AAPL]
	[DTree] : acc :  0.55, f1 : 0.56
	[RandomForest] : acc :  0.50, f1 : 0.50
	[SVM] : acc :  0.48, f1 : 0.53
	[LR] : acc :  0.45, f1 : 0.62


[AMZN]
	[DTree] : acc :  0.46, f1 : 0.47
	[RandomForest] : acc :  0.55, f1 : 0.71
	[SVM] : acc :  0.53, f1 : 0.64
	[LR] : acc :  0.54, f1 : 0.65


[CMCSA]
	[DTree] : acc :  0.51, f1 : 0.51
	[RandomForest] : acc :  0.53, f1 : 0.68
	[SVM] : acc :  0.53, f1 : 0.66
	[LR] : acc :  0.53, f1 : 0.69


[COST]
	[DTree] : acc :  0.52, f1 : 0.52
	[RandomForest] : acc :  0.48, f1 : 0.48
	[SVM] : acc :  0.48, f1 : 0.48
	[LR] : acc :  0.54, f1 : 0.70


[GM]
	[DTree] : acc :  0.53, f1 : 0.53
	[RandomForest] : acc :  0.49, f1 : 0.49
	[SVM] : acc :  0.47, f1 : 0.47
	[LR] : acc :  0.48, f1 : 0.48


[GOOG]
	[DTree] : acc :  0.50, f1 : 0.51
	[RandomForest] : acc :  0.54, f1 : 0.56
	[SVM] : acc :  0.47, f1 : 0.48
	[LR] 

In [22]:
for predict_n, quot_metrics in full_metrics.items():
    metrics_df = pd.DataFrame.from_dict(quot_metrics).T
    #print(metrics_df)
    acc_df = metrics_df.applymap(lambda metrics: metrics['acc'])
    f1_df = metrics_df.applymap(lambda metrics: metrics['f1'])
    
    plt.figure()
    acc_heatmap = sns.heatmap(acc_df, cmap ='mako', linewidths = 0.5, annot = True)
    acc_heatmap.figure.savefig(f"{METRICS_PATH}{predict_n}_acc.png")
    plt.close()
    
    plt.figure()
    f1_heatmap = sns.heatmap(f1_df, cmap ='mako', linewidths = 0.5, annot = True)
    f1_heatmap.figure.savefig(f"{METRICS_PATH}{predict_n}_f1.png")
    plt.close()